In [2]:
from obspy.taup import TauPyModel
model = TauPyModel(model="ak135irelandcrust")
from obspy.geodetics import degrees2kilometers, gps2dist_azimuth
from pyproj import Geod
geod = Geod(ellps="WGS84")
from obspy.geodetics import locations2degrees
import pandas as pd
import numpy as np
from multiprocessing import Pool
seisblast_files = '/home/bmelo/bin/SeisBlast/files'

In [3]:
stations = pd.read_csv(
    "/home/bmelo/bin/Quarry/James_files/Total_Stations_List.txt", 
    delimiter=' ',
    header = 0,
    names=['sta', 'name','lat', 'lon', 'elevation', 'net', 'end'])
quarries = pd.read_csv(
    '/home/bmelo/bin/Quarry/James_files/IRELAND.quarry.coords.txt',
    sep = r'\s+',
    header=None,
    names=['name','co','lat','lon'])
uniq_blasts = pd.read_csv('/home/bmelo/bin/SeisBlast/files/uniq_quarry-station_list.csv')
uniq_blasts

,closest-ev_id,quarry-lat,quarry-lon,origin_time,mag,sta,closest-time,count
0,dias2013kwhe,54.3797,-7.378597,2013-06-04 11:58:42.744,1.29,"['D34', 'DL31', 'IDGL', 'UCRUI']","[13.873, 10.696, 13.043, 16.183]",5
1,dias2013kwks,52.8995,-9.038942,2013-06-04 13:45:39.377,1.56,"['IA002', 'IA009', 'IAD01', 'UCAR3', 'UD10', '...","[27.592, 18.393, 3.43, 12.756, 35.399, 24.536,...",5
2,dias2013kwle,53.4341,-7.138017,2013-06-04 14:00:17.523,1.18,['UCD11'],[10.823],5
3,dias2013kwlj,52.5985,-7.200083,2013-06-04 14:05:07.330,1.16,"['DSB', 'UHELL']","[15.376, 15.765]",5
4,dias2013kwls,53.3092,-9.007666,2013-06-04 14:16:21.183,1.71,"['DSB', 'IA009', 'IA013', 'IAD01', 'IGLA', 'UC...","[28.79, 10.624, 21.342, 10.326, 4.683, 10.139,...",5
...,...,...,...,...,...,...,...,...
1175,dias2014yqpq,54.8313,-6.050479,2014-12-17 14:56:14.298,1.26,['DL14'],[19.989],3
1176,dias2014ysgw,53.2730,-6.502024,2014-12-18 12:46:48.262,0.76,['ILTH'],[14.815],5
1177,dias2014yshe,54.2518,-6.586716,2014-12-18 12:55:53.479,0.99,['IA003'],[26.231],5
1178,dias2014yucm,54.5246,-8.088243,2014-12-19 12:50:04.704,1.09,"['DL13', 'DL24', 'IA009', 'IDGL', 'ILTH', 'UD0...","[14.473, 13.673, 15.818, 12.152, 22.142, 7.722...",5


In [4]:
def get_ray_info(index_src):
    """
    Computes and plots ray paths for a given source and returns ray path data.
    
    Parameters:
    index_src (int): Index of the quarry in the dataframe.
    type (str): Type of plot ('map' or 'cross-section').
    ax (pygmt.Figure or matplotlib.Axes): Plotting object.
    
    Returns:
    pd.DataFrame: DataFrame containing ray path latitude, longitude, and depth.
    """
    quarry = uniq_blasts.iloc[index_src]
    q_lat, q_lon = quarry['quarry-lat'], quarry['quarry-lon']
    st_list = eval(quarry['sta'])
    ray_data = pd.DataFrame()
    ray_data_list = []
    # Loop over stations
    for st_id in st_list:
        station_info = stations.loc[stations['name'] == st_id].squeeze()
        # Compute azimuth and epicentral distance
        _, azimuth, _ = gps2dist_azimuth(
            q_lat, 
            q_lon, 
            station_info['lat'], 
            station_info['lon'])
        dist_in_deg = locations2degrees(
            q_lat,
            q_lon,
            station_info['lat'],
            station_info['lon'])
        # Get ray paths from TauP
        arrivals = model.get_ray_paths(
            0,  # source depth
            dist_in_deg,  # distance in degrees
            ["P"],  # phase
            0)  # station elevation?

        if arrivals:
            arrival = arrivals[0]
            # Create ray path data for the current station
            ray_data_current = pd.DataFrame({
                'event-ID': [quarry['closest-ev_id']],
                'station': [st_id],
                'station_lon': [station_info['lon']],
                'station_lat': [station_info['lat']],
                'quarry_lon': [q_lon],
                'quarry_lat': [q_lat],
                'lon': [list(q_lon + (np.rad2deg(arrival.path['dist']) * np.sin(np.radians(azimuth))) / np.cos(np.radians(q_lat)))],
                'lat': [list(q_lat + np.rad2deg(arrival.path['dist']) * np.cos(np.radians(azimuth)))],
                'depth': [list(-arrival.path['depth'])]
            })
            # Concatenate with the main ray data DataFrame
            #ray_data = pd.concat([ray_data, ray_data_current], ignore_index=True)
            ray_data_list.append(ray_data_current)
    
    if ray_data_list:
        return pd.concat(ray_data_list, ignore_index=True)
    return pd.DataFrame()

In [5]:
# # Process all list
n_cores = 100
ray_info = pd.DataFrame()

# Create the pool and process in parallel
with Pool(n_cores) as pool:
    ray_info_list = pool.map(get_ray_info, range(len(uniq_blasts)))  # or range(len(uniq_blasts))
    
# Remove any empty DataFrames (if any station had no arrivals)
ray_info_list = [df for df in ray_info_list if not df.empty]

# Concatenate all at once (much faster than incremental concat)
ray_info = pd.concat(ray_info_list, ignore_index=True)

# Save ray path data to CSV
ray_info.to_csv(
    "/home/bmelo/bin/SeisBlast/files/ray_path_data.csv", 
    index=False, 
    sep=' ')

ray_info.to_pickle("/home/bmelo/bin/SeisBlast/files/ray_path_data.pkl")